In [8]:
import torch
import torch.nn as nn
from torchvision.ops import StochasticDepth



In [3]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_dim, reduction_ratio=16, use_residual=False) -> None:
        super(SqueezeExcitation, self).__init__()

        self.use_residual = use_residual

        self.squeeze = nn.AdaptiveAvgPool2d(1)

        self.excitation = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=in_dim//reduction_ratio, kernel_size=1, stride=1),
            nn.SiLU(), # nn.ReLU()
            nn.Conv2d(in_channels=in_dim//reduction_ratio, out_channels=in_dim, kernel_size=1, stride=1),
            nn.Sigmoid()
        )

    def forward(self, x):

        se_out = self.squeeze(x)
        se_out = self.excitation(se_out)
        se_out = se_out * x 

        if self.use_residual:
            se_out += x
        
        return se_out

In [4]:
class MBConv(nn.Module):
    def __init__(self, in_dim, out_dim, expasion_ratio=4, squeeze_ratio=4, kernel_size=3, stride=1) -> None:
        super(MBConv, self).__init__()
        self.use_residual = in_dim == out_dim and stride == 1
        hidden_dim = int ( in_dim * expasion_ratio)
        padding = kernel_size // 2  # if kernel size 3 -> padding 1, kerenl size 5 -> padding 2

        self.expasion = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=hidden_dim, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU()
        )

        self.dwise = nn.Sequential(
            nn.Conv2d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=kernel_size, stride=stride, padding=padding, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU()
        )

        self.se = SqueezeExcitation(in_dim,hidden_dim, reduction_ratio=squeeze_ratio)

        self.projection = nn.Sequential(
            nn.Conv2d(in_channels=hidden_dim, out_channels=out_dim, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_dim)
        )

    def forward(self, x):

        h = self.expasion(x)
        h = self.dwise(h)
        h = self.se(h)
        h = self.projection(h)

        if self.use_residual:
            h = h + x 

        return h
        

In [5]:
class FusedMBConv(nn.Module):
    def __init__(self, in_dim, out_dim, expasion_ratio=4, kernel_size=3, stride=1) -> None:
        super(MBConv, self).__init__()
        self.use_residual = in_dim == out_dim and stride == 1
        hidden_dim = int ( in_dim * expasion_ratio)
        padding = kernel_size // 2  # if kernel size 3 -> padding 1, kerenl size 5 -> padding 2

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=hidden_dim, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU()
        )

        self.se = SqueezeExcitation(hidden_dim,hidden_dim, reduction_ratio=16)

        self.projection = nn.Sequential(
            nn.Conv2d(in_channels=hidden_dim, out_channels=out_dim, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_dim)
        )

    def forward(self, x):

        h = self.conv(x)
        h = self.se(h)
        h = self.projection(h)

        if self.use_residual:
            h = h + x 

        return h
        

In [6]:
# efficientNetV2_S model 
class efficientNetV2(nn.Module):
    def __init__(self, num_classes=1000) -> None:
        super(efficientNetV2, self).__init__()

        hidden_dim = [24, 24, 48, 64, 128, 160, 256, 1280]

        self.init_conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=hidden_dim[0], kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU()
        )
        
        # c 24 -> 24
        self.conv1 = nn.Sequential(
            *([FusedMBConv(in_dim=hidden_dim[0], out_dim=hidden_dim[1], expasion_ratio=1, stride=1) for _ in range(1)]
              + [FusedMBConv(in_dim=hidden_dim[1], out_dim=hidden_dim[1], expasion_ratio=1, stride=1)])
        )

        # c 24 -> 48
        self.conv2 = nn.Sequential(
            *([FusedMBConv(in_dim=hidden_dim[1], out_dim=hidden_dim[2], expasion_ratio=4, kernel_size=3, stride=2)]
            + [FusedMBConv(in_dim=hidden_dim[2], out_dim=hidden_dim[2], expasion_ratio=4, kernel_size=3, stride=1) for _ in range(3)])
        )

        # c 48 -> 64
        self.conv3 = nn.Sequential(
            *([FusedMBConv(in_dim=hidden_dim[2], out_dim=hidden_dim[3], expasion_ratio=4, kernel_size=3, stride=2)]
            + [FusedMBConv(in_dim=hidden_dim[3], out_dim=hidden_dim[3], expasion_ratio=4, kernel_size=3, stride=1) for _ in range(3)])
        )

        # c 64 -> 128
        self.conv4 = nn.Sequential(
            *([MBConv(in_dim=hidden_dim[3], out_dim=hidden_dim[4], expasion_ratio=4, squeeze_ratio=4, kernel_size=3, stride=2)]
            + [MBConv(in_dim=hidden_dim[4], out_dim=hidden_dim[4], expasion_ratio=4, squeeze_ratio=4, kernel_size=3, stride=1) for _ in range(5)])
        )

        # c 128 -> 160
        self.conv5 = nn.Sequential(
            *([MBConv(in_dim=hidden_dim[4], out_dim=hidden_dim[5], expasion_ratio=6, squeeze_ratio=4, kernel_size=3, stride=1)]
            + [MBConv(in_dim=hidden_dim[5], out_dim=hidden_dim[5], expasion_ratio=6, squeeze_ratio=4, kernel_size=3, stride=1) for _ in range(8)])
        )

        # c 160 -> 256
        self.conv6 = nn.Sequential(
            *([MBConv(in_dim=hidden_dim[5], out_dim=hidden_dim[6], expasion_ratio=6, squeeze_ratio=4, kernel_size=3, stride=2)]
            + [MBConv(in_dim=hidden_dim[6], out_dim=hidden_dim[6], expasion_ratio=6, squeeze_ratio=4, kernel_size=3, stride=1) for _ in range(14)])
        )

        # c 256 -> 1280
        self.last_conv = nn.Sequential(
            nn.Conv2d(in_channels=hidden_dim[6], out_channels=hidden_dim[7], kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.SiLU()
        )

        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Conv2d(in_channels=hidden_dim[7], out_channels=num_classes, kernel_size=1, stride=1)
    
    def forward(self, x):

        h = self.init_conv(x)

        h = self.conv1(h)
        h = self.conv2(h)
        h = self.conv3(h)
        h = self.conv4(h)
        h = self.conv5(h)
        h = self.conv6(h)

        h = self.last_conv(h)

        p = self.pooling(h)

        out = self.fc(p)

        return out





In [ ]:
# progressive learning example

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define the transformations to be applied to the input images
transform_train = transforms.Compose([
    transforms.Resize((256, 256)), # resize the input images to 256x256
    transforms.RandomCrop((224, 224)), # crop the input images to 224x224 randomly
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((256, 256)), # resize the input images to 256x256
    transforms.CenterCrop((224, 224)), # crop the input images to 224x224 at the center
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets
train_dataset = datasets.ImageFolder("train/", transform=transform_train)
val_dataset = datasets.ImageFolder("val/", transform=transform_val)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.2) # initial value of dropout probability

        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)

        return x
    
# Instantiate model
model = MyModel(input_dim=224*224*3, hidden_dim=100, output_dim=2)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Train model
for epoch in range(10):
    # Increase image size
    if epoch == 3:
        transform_train = transforms.Compose([
            transforms.Resize((512, 512)), # resize the input images to 512x512
            transforms.RandomCrop((448, 448)), # crop the input images to 448x448 randomly
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        train_dataset = datasets.ImageFolder("train/", transform=transform_train)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

    # Train for one epoch
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        inputs = inputs.view(inputs.size(0), -1) # flatten the input images
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    with torch.no_grad():
        val_loss = 0.0
        val_acc = 0.0
        for inputs, labels in val_dataloader:
            inputs = inputs.view(inputs.size(0), -1) # flatten the input images
            outputs = model(inputs)
            val_loss += criterion(outputs, labels)
            val_acc += (outputs.argmax(dim=1) == labels).sum().item()
        val_loss /= len(val_dataloader)